In [1]:
# Test the several architecture of ProtoNN versus the K-NN algorithm

# import ProtoNN code
import protoNN as protograph
import protoNNTrainer as prototrainer
import utils as utils
# import general modules
import numpy as np
import torch
import math
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
import gc

# need to change the dataset location
X, Y = utils.handleUSPS('../../../data_set/usps.h5')

dataset = utils.CustomDatasetProtoNN(X, Y, rescaling=True, binary=False)

C:\Users\pietr\anaconda3\envs\edgeml\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset input rescaled
Dataset loaded, input space: (7291, 256) , output space: (7291, 1)
Dataset reshaped, input space: (7291, 256) , output space: (7291, 10)


In [2]:
# Stratified k fold
k_folds = 5 # default 5 or 10
stratifiedk_fold = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=1)
knn = KNeighborsClassifier(n_neighbors = 13)
print("Stratified KFOLD INITIALIZED")

Stratified KFOLD INITIALIZED


In [3]:
# fixed ProtoNN HYPERPARAMETERS
# regularizer of W, B and Z: HYPERPARAMETER
regW, regB, regZ = 0, 0, 0

# sparsity of W, B and Z: HYPERPARAMETER, CONSTRAINTS OF MEMORY
sparcityW, sparcityB, sparcityZ = 1, 1, 1

# initial learning rate of the OPTIMIZER: HYPERPARAMETER
learningRate = 0.1
# lossType: 'l2'
lossType= 'l2'

# TRAINING PHASE
# batchsize: HAS TO BE DEFINED
batchsize = 100
# epochs: HAS TO BE DEFINED
epochs = 100
# printStep: HAS TO BE DEFINED
printStep = 100
# valStep: HAS TO BE DEFINED
valStep = 100

# print virables of ProtoNN training
print("\nbatch size:\t", batchsize,
      "\nepochs:\t\t", epochs,
      "\nprint step:\t", printStep,
      "\nevaluation step:", valStep)


batch size:	 100 
epochs:		 100 
print step:	 100 
evaluation step: 100


In [4]:
# FOR 2 KB
size = [2]
size_parameters = [[7, 10]] # dim and proto
test_case = 0;
for e in size_parameters:
    test_case += 1

# For fold results
results = np.zeros([k_folds, test_case])
structure = np.zeros([test_case, 2])
# structure [0]: numProto, [1]: prjdim

# Start print
print('--------------------------------')
# K-fold Cross Validation model evaluation
for fold, (train_ids, test_ids) in enumerate(stratifiedk_fold.split(dataset.data, Y)): # give y is just for compatibility in KFOLD

    # Print
    print(f'FOLD {fold}')
    print('--------------------------------')
    
    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
    
    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=train_ids.shape[0],
                      sampler=train_subsampler)
    testloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=test_ids.shape[0],
                      sampler=test_subsampler)

    train_inputs, train_targets = next(iter(trainloader))
    test_inputs, test_targets = next(iter(testloader))

    # SET FIXED DIMENSION
    input_dimension = train_inputs.shape[1]
    numOutputLabels = dataset.numClasses
    
    index_result = 0
    for index_size, couple_elem in enumerate(size_parameters):
        # projection_dimension: HYPERPARAMETER
        projection_dimension = couple_elem[0]
        # numPrototypes: HYPERPARAMETR
        numPrototypes = couple_elem[1]
        
        structure[index_size][0] = numPrototypes
        structure[index_size][1] = projection_dimension
        
        # print structure of ProtoNN
        print("num prototypes:\t", numPrototypes,
              "\nporjection dim:\t", projection_dimension)
        
        # sparsity of W, B and Z SPECIFIC FOR THIS COMBINATION OF PARAMS
        sparcityW, sparcityB, sparcityZ = 0.1, 1, 0.3
        
        # initialize W as random - to use random seed fixed for testing
        W = np.random.rand(input_dimension, projection_dimension) 
        
        # initialize protoNN
        protoNNmodel = protograph.ProtoNN(input_dimension, projection_dimension,
                                     numPrototypes, numOutputLabels,
                                     gamma=0.05, W=W)
        protoNNmodel.initializePrototypesAndGamma(train_inputs, train_targets,
                                     input_W=W, overrideGamma=True)
        protoNNtrainer = prototrainer.ProtoNNTrainer(protoNNmodel,
                                     sparcityW, sparcityB, sparcityZ,
                                     regW, regB, regZ,
                                     learningRate,
                                     lossType)
        print("Model size:", protoNNtrainer.getModelSize(), "KB")
        current_size = protoNNtrainer.getModelSize()
        result_dic = protoNNtrainer.train(
                              train_inputs.float(),
                              test_inputs.float(),
                              train_targets.float(),
                              test_targets.float(),
                              batchsize,
                              epochs,
                              printStep,
                              valStep,
                              verbose=False, history=False)
        correct = result_dic.get('correctPredictions').item()
        total  = result_dic.get('totalPredictions')
        accuracy = 100 * correct / total
        
        # Print accuracy
        print('Accuracy for fold %d with m = %f,d = %d: %d %%' % (fold, numPrototypes, projection_dimension, accuracy))
        results[fold][index_result] = accuracy
        index_result+=1
        del protoNNmodel
        del protoNNtrainer
        gc.collect()

    print('--------------------------------')

# Print fold results
print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds}')
print('--------------------------------')
results = np.sum(results, axis = 0) / k_folds
print("Average Accuracy")
for i in range (0, test_case):
    print("ProtoNN with size", size[0], "KB, prototype", structure[i][0], ", prj dim", structure[i][1],", Accuracy: \t:", results[i])

--------------------------------
FOLD 0
--------------------------------
num prototypes:	 10 
porjection dim:	 7
Model size: 1.9536000000000002 KB
Accuracy for fold 0 with m = 10.000000,d = 7: 82 %
--------------------------------
FOLD 1
--------------------------------
num prototypes:	 10 
porjection dim:	 7
Model size: 1.9536000000000002 KB
Accuracy for fold 1 with m = 10.000000,d = 7: 90 %
--------------------------------
FOLD 2
--------------------------------
num prototypes:	 10 
porjection dim:	 7
Model size: 1.9536000000000002 KB
Accuracy for fold 2 with m = 10.000000,d = 7: 82 %
--------------------------------
FOLD 3
--------------------------------
num prototypes:	 10 
porjection dim:	 7
Model size: 1.9536000000000002 KB
Accuracy for fold 3 with m = 10.000000,d = 7: 87 %
--------------------------------
FOLD 4
--------------------------------
num prototypes:	 10 
porjection dim:	 7
Model size: 1.9536000000000002 KB
Accuracy for fold 4 with m = 10.000000,d = 7: 80 %
----------

In [5]:
# FOR 4 KB
size = [4]
size_parameters = [[7, 16]] # dim and proto
test_case = 0;
for e in size_parameters:
    test_case += 1

# For fold results
results = np.zeros([k_folds, test_case])
structure = np.zeros([test_case, 2])
# structure [0]: numProto, [1]: prjdim

# Start print
print('--------------------------------')
# K-fold Cross Validation model evaluation
for fold, (train_ids, test_ids) in enumerate(stratifiedk_fold.split(dataset.data, Y)): # give y is just for compatibility in KFOLD

    # Print
    print(f'FOLD {fold}')
    print('--------------------------------')
    
    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
    
    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=train_ids.shape[0],
                      sampler=train_subsampler)
    testloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=test_ids.shape[0],
                      sampler=test_subsampler)

    train_inputs, train_targets = next(iter(trainloader))
    test_inputs, test_targets = next(iter(testloader))

    # SET FIXED DIMENSION
    input_dimension = train_inputs.shape[1]
    numOutputLabels = dataset.numClasses
    
    index_result = 0
    for index_size, couple_elem in enumerate(size_parameters):
        # projection_dimension: HYPERPARAMETER
        projection_dimension = couple_elem[0]
        # numPrototypes: HYPERPARAMETR
        numPrototypes = couple_elem[1]
        
        structure[index_size][0] = numPrototypes
        structure[index_size][1] = projection_dimension
        
        # print structure of ProtoNN
        print("num prototypes:\t", numPrototypes,
              "\nporjection dim:\t", projection_dimension)
        
        # sparsity of W, B and Z SPECIFIC FOR THIS COMBINATION OF PARAMS
        sparcityW, sparcityB, sparcityZ = 0.2, 1, 0.3
        
        # initialize W as random - to use random seed fixed for testing
        W = np.random.rand(input_dimension, projection_dimension) 
        
        # initialize protoNN
        protoNNmodel = protograph.ProtoNN(input_dimension, projection_dimension,
                                     numPrototypes, numOutputLabels,
                                     gamma=0.05, W=W)
        protoNNmodel.initializePrototypesAndGamma(train_inputs, train_targets,
                                     input_W=W, overrideGamma=True)
        protoNNtrainer = prototrainer.ProtoNNTrainer(protoNNmodel,
                                     sparcityW, sparcityB, sparcityZ,
                                     regW, regB, regZ,
                                     learningRate,
                                     lossType)
        print("Model size:", protoNNtrainer.getModelSize(), "KB")
        current_size = protoNNtrainer.getModelSize()
        result_dic = protoNNtrainer.train(
                              train_inputs.float(),
                              test_inputs.float(),
                              train_targets.float(),
                              test_targets.float(),
                              batchsize,
                              epochs,
                              printStep,
                              valStep,
                              verbose=False, history=False)
        correct = result_dic.get('correctPredictions').item()
        total  = result_dic.get('totalPredictions')
        accuracy = 100 * correct / total
        
        # Print accuracy
        print('Accuracy for fold %d with m = %f,d = %d: %d %%' % (fold, numPrototypes, projection_dimension, accuracy))
        results[fold][index_result] = accuracy
        index_result+=1
        del protoNNmodel
        del protoNNtrainer
        gc.collect()

    print('--------------------------------')

# Print fold results
print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds}')
print('--------------------------------')
results = np.sum(results, axis = 0) / k_folds
print("Average Accuracy")
for i in range (0, test_case):
    print("ProtoNN with size", size[0], "KB, prototype", structure[i][0], ", prj dim", structure[i][1],", Accuracy: \t:", results[i])

--------------------------------
FOLD 0
--------------------------------
num prototypes:	 16 
porjection dim:	 7
Model size: 3.6992000000000003 KB
Accuracy for fold 0 with m = 16.000000,d = 7: 88 %
--------------------------------
FOLD 1
--------------------------------
num prototypes:	 16 
porjection dim:	 7
Model size: 3.6992000000000003 KB
Accuracy for fold 1 with m = 16.000000,d = 7: 93 %
--------------------------------
FOLD 2
--------------------------------
num prototypes:	 16 
porjection dim:	 7
Model size: 3.6992000000000003 KB
Accuracy for fold 2 with m = 16.000000,d = 7: 89 %
--------------------------------
FOLD 3
--------------------------------
num prototypes:	 16 
porjection dim:	 7
Model size: 3.6992000000000003 KB
Accuracy for fold 3 with m = 16.000000,d = 7: 91 %
--------------------------------
FOLD 4
--------------------------------
num prototypes:	 16 
porjection dim:	 7
Model size: 3.6992000000000003 KB
Accuracy for fold 4 with m = 16.000000,d = 7: 91 %
----------

In [6]:
# FOR SMALL SIZE
size = [2, 4, 8]

# For fold results
test_case = len(size)
results = np.zeros([k_folds, test_case])
structure = np.zeros([test_case, 2])

# Start print
print('--------------------------------')
# K-fold Cross Validation model evaluation
for fold, (train_ids, test_ids) in enumerate(stratifiedk_fold.split(dataset.data, Y)): # give y is just for compatibility in KFOLD

    # Print
    print(f'FOLD {fold}')
    print('--------------------------------')
    
    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
    
    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=train_ids.shape[0],
                      sampler=train_subsampler)
    testloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=test_ids.shape[0],
                      sampler=test_subsampler)

    train_inputs, train_targets = next(iter(trainloader))
    test_inputs, test_targets = next(iter(testloader))
    
    # SET FIXED DIMENSION
    input_dimension = train_inputs.shape[1]
    numOutputLabels = dataset.numClasses
    
    index_result = 0
    for index_size, size_limit in enumerate(size):
        
        # projection_dimension: HYPERPARAMETER
        projection_dimension = math.trunc(utils.max_projection_dimensions(size[index_size], numOutputLabels, input_dimension, numOutputLabels))
        # numPrototypes: HYPERPARAMETR
        numPrototypes = math.trunc(utils.max_number_prototypes(size[index_size], projection_dimension, input_dimension, numOutputLabels))
        structure[index_size][0] = numPrototypes
        structure[index_size][1] = projection_dimension
        
        # print structure of ProtoNN
        print("num prototypes:\t", numPrototypes,
              "\nporjection dim:\t", projection_dimension)
        
        # initialize W as random - to use random seed fixed for testing
        W = np.random.rand(input_dimension, projection_dimension) 
        
        # sparsity of W, B and Z SPECIFIC FOR THIS COMBINATION OF PARAMS
        sparcityW, sparcityB, sparcityZ = 1, 1, 1
        
        # initialize protoNN
        protoNNmodel = protograph.ProtoNN(input_dimension, projection_dimension,
                                     numPrototypes, numOutputLabels,
                                     gamma=0.05, W=W)
        protoNNmodel.initializePrototypesAndGamma(train_inputs, train_targets,
                                     input_W=W, overrideGamma=True)
        protoNNtrainer = prototrainer.ProtoNNTrainer(protoNNmodel,
                                     sparcityW, sparcityB, sparcityZ,
                                     regW, regB, regZ,
                                     learningRate,
                                     lossType)
        print("Model size:", protoNNtrainer.getModelSize(), "KB")
        current_size = protoNNtrainer.getModelSize()
        result_dic = protoNNtrainer.train(
                              train_inputs.float(),
                              test_inputs.float(),
                              train_targets.float(),
                              test_targets.float(),
                              batchsize,
                              epochs,
                              printStep,
                              valStep,
                              verbose=False, history=False)
        correct = result_dic.get('correctPredictions').item()
        total  = result_dic.get('totalPredictions')
        accuracy = 100 * correct / total

        # Print accuracy
        print('Accuracy for fold %d with m = %f,d = %d: %d %%' % (fold, numPrototypes, projection_dimension, accuracy))
        results[fold][index_result] = accuracy
        index_result+=1
        del protoNNmodel
        del protoNNtrainer
        gc.collect()

    print('--------------------------------')

# Print fold results
print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds}')
print('--------------------------------')
results = np.sum(results, axis = 0) / k_folds
print("Average Accuracy")
for i in range (0, test_case):
    print("ProtoNN with size", size[0], "KB, prototype", structure[i][0], ", prj dim", structure[i][1],", Accuracy: \t:", results[i])

--------------------------------
FOLD 0
--------------------------------
num prototypes:	 22 
porjection dim:	 1
Model size: 1.992 KB
Accuracy for fold 0 with m = 22.000000,d = 1: 50 %
num prototypes:	 17 
porjection dim:	 3
Model size: 3.956 KB
Accuracy for fold 0 with m = 17.000000,d = 3: 86 %
num prototypes:	 12 
porjection dim:	 7
Model size: 7.984 KB
Accuracy for fold 0 with m = 12.000000,d = 7: 93 %
--------------------------------
FOLD 1
--------------------------------
num prototypes:	 22 
porjection dim:	 1
Model size: 1.992 KB
Accuracy for fold 1 with m = 22.000000,d = 1: 48 %
num prototypes:	 17 
porjection dim:	 3
Model size: 3.956 KB
Accuracy for fold 1 with m = 17.000000,d = 3: 89 %
num prototypes:	 12 
porjection dim:	 7
Model size: 7.984 KB
Accuracy for fold 1 with m = 12.000000,d = 7: 93 %
--------------------------------
FOLD 2
--------------------------------
num prototypes:	 22 
porjection dim:	 1
Model size: 1.992 KB
Accuracy for fold 2 with m = 22.000000,d = 1: 48

In [7]:
# FOR 16 KB
size = [16]
size_parameters = [[14, 17], [10, 72], [10, 50], [12, 40]] # dim and proto
test_case = 0;
for e in size_parameters:
    test_case += 1

# For fold results
results = np.zeros([k_folds, test_case])
structure = np.zeros([test_case, 2])
# structure [0]: numProto, [1]: prjdim

# Start print
print('--------------------------------')
# K-fold Cross Validation model evaluation
for fold, (train_ids, test_ids) in enumerate(stratifiedk_fold.split(dataset.data, Y)): # give y is just for compatibility in KFOLD

    # Print
    print(f'FOLD {fold}')
    print('--------------------------------')
    
    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
    
    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=train_ids.shape[0],
                      sampler=train_subsampler)
    testloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=test_ids.shape[0],
                      sampler=test_subsampler)

    train_inputs, train_targets = next(iter(trainloader))
    test_inputs, test_targets = next(iter(testloader))

    # SET FIXED DIMENSION
    input_dimension = train_inputs.shape[1]
    numOutputLabels = dataset.numClasses
    
    index_result = 0
    for index_size, couple_elem in enumerate(size_parameters):
        # projection_dimension: HYPERPARAMETER
        projection_dimension = couple_elem[0]
        # numPrototypes: HYPERPARAMETR
        numPrototypes = couple_elem[1]
        
        structure[index_size][0] = numPrototypes
        structure[index_size][1] = projection_dimension
        
        # print structure of ProtoNN
        print("num prototypes:\t", numPrototypes,
              "\nporjection dim:\t", projection_dimension)
        
        # sparsity of W, B and Z SPECIFIC FOR THIS COMBINATION OF PARAMS
        sparcityW, sparcityB, sparcityZ = 1, 1, 1
        
        # initialize W as random - to use random seed fixed for testing
        W = np.random.rand(input_dimension, projection_dimension) 
        
        # initialize protoNN
        protoNNmodel = protograph.ProtoNN(input_dimension, projection_dimension,
                                     numPrototypes, numOutputLabels,
                                     gamma=0.05, W=W)
        protoNNmodel.initializePrototypesAndGamma(train_inputs, train_targets,
                                     input_W=W, overrideGamma=True)
        protoNNtrainer = prototrainer.ProtoNNTrainer(protoNNmodel,
                                     sparcityW, sparcityB, sparcityZ,
                                     regW, regB, regZ,
                                     learningRate,
                                     lossType)
        print("Model size:", protoNNtrainer.getModelSize(), "KB")
        current_size = protoNNtrainer.getModelSize()
        result_dic = protoNNtrainer.train(
                              train_inputs.float(),
                              test_inputs.float(),
                              train_targets.float(),
                              test_targets.float(),
                              batchsize,
                              epochs,
                              printStep,
                              valStep,
                              verbose=False, history=False)
        correct = result_dic.get('correctPredictions').item()
        total  = result_dic.get('totalPredictions')
        accuracy = 100 * correct / total
        
        # Print accuracy
        print('Accuracy for fold %d with m = %f,d = %d: %d %%' % (fold, numPrototypes, projection_dimension, accuracy))
        results[fold][index_result] = accuracy
        index_result+=1
        del protoNNmodel
        del protoNNtrainer
        gc.collect()

    print('--------------------------------')

# Print fold results
print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds}')
print('--------------------------------')
results = np.sum(results, axis = 0) / k_folds
print("Average Accuracy")
for i in range (0, test_case):
    print("ProtoNN with size", size[0], "KB, prototype", structure[i][0], ", prj dim", structure[i][1],", Accuracy: \t:", results[i])

--------------------------------
FOLD 0
--------------------------------
num prototypes:	 17 
porjection dim:	 14
Model size: 15.968 KB
Accuracy for fold 0 with m = 17.000000,d = 14: 94 %
num prototypes:	 72 
porjection dim:	 10
Model size: 16.0 KB
Accuracy for fold 0 with m = 72.000000,d = 10: 94 %
num prototypes:	 50 
porjection dim:	 10
Model size: 14.24 KB
Accuracy for fold 0 with m = 50.000000,d = 10: 93 %
num prototypes:	 40 
porjection dim:	 12
Model size: 15.808 KB
Accuracy for fold 0 with m = 40.000000,d = 12: 94 %
--------------------------------
FOLD 1
--------------------------------
num prototypes:	 17 
porjection dim:	 14
Model size: 15.968 KB
Accuracy for fold 1 with m = 17.000000,d = 14: 94 %
num prototypes:	 72 
porjection dim:	 10
Model size: 16.0 KB
Accuracy for fold 1 with m = 72.000000,d = 10: 93 %
num prototypes:	 50 
porjection dim:	 10
Model size: 14.24 KB
Accuracy for fold 1 with m = 50.000000,d = 10: 93 %
num prototypes:	 40 
porjection dim:	 12
Model size: 15

In [8]:
# FOR 32 KB
size = [32]
size_parameters = [[25, 40], [15, 160], [15, 50], [20, 50]] # dim and proto
test_case = 0;
for e in size_parameters:
    test_case += 1

# For fold results
results = np.zeros([k_folds, test_case])
structure = np.zeros([test_case, 2])
# structure [0]: numProto, [1]: prjdim

# Start print
print('--------------------------------')
# K-fold Cross Validation model evaluation
for fold, (train_ids, test_ids) in enumerate(stratifiedk_fold.split(dataset.data, Y)): # give y is just for compatibility in KFOLD

    # Print
    print(f'FOLD {fold}')
    print('--------------------------------')
    
    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
    
    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=train_ids.shape[0],
                      sampler=train_subsampler)
    testloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=test_ids.shape[0],
                      sampler=test_subsampler)

    train_inputs, train_targets = next(iter(trainloader))
    test_inputs, test_targets = next(iter(testloader))

    # SET FIXED DIMENSION
    input_dimension = train_inputs.shape[1]
    numOutputLabels = dataset.numClasses
    
    index_result = 0
    for index_size, couple_elem in enumerate(size_parameters):
        # projection_dimension: HYPERPARAMETER
        projection_dimension = couple_elem[0]
        # numPrototypes: HYPERPARAMETR
        numPrototypes = couple_elem[1]
        
        structure[index_size][0] = numPrototypes
        structure[index_size][1] = projection_dimension
        
        # print structure of ProtoNN
        print("num prototypes:\t", numPrototypes,
              "\nporjection dim:\t", projection_dimension)
        
        # initialize W as random - to use random seed fixed for testing
        W = np.random.rand(input_dimension, projection_dimension)
        
        # sparsity of W, B and Z SPECIFIC FOR THIS COMBINATION OF PARAMS
        sparcityW, sparcityB, sparcityZ = 1, 1, 1
        
        # initialize protoNN
        protoNNmodel = protograph.ProtoNN(input_dimension, projection_dimension,
                                     numPrototypes, numOutputLabels,
                                     gamma=0.05, W=W)
        protoNNmodel.initializePrototypesAndGamma(train_inputs, train_targets,
                                     input_W=W, overrideGamma=True)
        protoNNtrainer = prototrainer.ProtoNNTrainer(protoNNmodel,
                                     sparcityW, sparcityB, sparcityZ,
                                     regW, regB, regZ,
                                     learningRate,
                                     lossType)
        print("Model size:", protoNNtrainer.getModelSize(), "KB")
        current_size = protoNNtrainer.getModelSize()
        result_dic = protoNNtrainer.train(
                              train_inputs.float(),
                              test_inputs.float(),
                              train_targets.float(),
                              test_targets.float(),
                              batchsize,
                              epochs,
                              printStep,
                              valStep,
                              verbose=False, history=False)
        correct = result_dic.get('correctPredictions').item()
        total  = result_dic.get('totalPredictions')
        accuracy = 100 * correct / total
        
        # Print accuracy
        print('Accuracy for fold %d with m = %f,d = %d: %d %%' % (fold, numPrototypes, projection_dimension, accuracy))
        results[fold][index_result] = accuracy
        index_result+=1
        del protoNNmodel
        del protoNNtrainer
        gc.collect()

    print('--------------------------------')

# Print fold results
print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds}')
print('--------------------------------')
results = np.sum(results, axis = 0) / k_folds
print("Average Accuracy")
for i in range (0, test_case):
    print("ProtoNN with size", size[0], "KB, prototype", structure[i][0], ", prj dim", structure[i][1],", Accuracy: \t:", results[i])

--------------------------------
FOLD 0
--------------------------------
num prototypes:	 40 
porjection dim:	 25
Model size: 31.2 KB
Accuracy for fold 0 with m = 40.000000,d = 25: 94 %
num prototypes:	 160 
porjection dim:	 15
Model size: 31.36 KB
Accuracy for fold 0 with m = 160.000000,d = 15: 94 %
num prototypes:	 50 
porjection dim:	 15
Model size: 20.36 KB
Accuracy for fold 0 with m = 50.000000,d = 15: 94 %
num prototypes:	 50 
porjection dim:	 20
Model size: 26.48 KB
Accuracy for fold 0 with m = 50.000000,d = 20: 94 %
--------------------------------
FOLD 1
--------------------------------
num prototypes:	 40 
porjection dim:	 25
Model size: 31.2 KB
Accuracy for fold 1 with m = 40.000000,d = 25: 94 %
num prototypes:	 160 
porjection dim:	 15
Model size: 31.36 KB
Accuracy for fold 1 with m = 160.000000,d = 15: 93 %
num prototypes:	 50 
porjection dim:	 15
Model size: 20.36 KB
Accuracy for fold 1 with m = 50.000000,d = 15: 94 %
num prototypes:	 50 
porjection dim:	 20
Model size: 2

In [9]:
# FOR 64 KB
size = [64]
size_parameters = [[50, 50], [15, 480], [15,  240], [15,50]] # dim and proto
test_case = 0;
for e in size_parameters:
    test_case += 1

# For fold results
results = np.zeros([k_folds, test_case])
structure = np.zeros([test_case, 2])
# structure [0]: numProto, [1]: prjdim

# Start print
print('--------------------------------')
# K-fold Cross Validation model evaluation
for fold, (train_ids, test_ids) in enumerate(stratifiedk_fold.split(dataset.data, Y)): # give y is just for compatibility in KFOLD

    # Print
    print(f'FOLD {fold}')
    print('--------------------------------')
    
    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
    
    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=train_ids.shape[0],
                      sampler=train_subsampler)
    testloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=test_ids.shape[0],
                      sampler=test_subsampler)

    train_inputs, train_targets = next(iter(trainloader))
    test_inputs, test_targets = next(iter(testloader))

    # SET FIXED DIMENSION
    input_dimension = train_inputs.shape[1]
    numOutputLabels = dataset.numClasses
    
    index_result = 0
    for index_size, couple_elem in enumerate(size_parameters):
        # projection_dimension: HYPERPARAMETER
        projection_dimension = couple_elem[0]
        # numPrototypes: HYPERPARAMETR
        numPrototypes = couple_elem[1]
        
        structure[index_size][0] = numPrototypes
        structure[index_size][1] = projection_dimension
        
        # print structure of ProtoNN
        print("num prototypes:\t", numPrototypes,
              "\nporjection dim:\t", projection_dimension)
        
        # sparsity of W, B and Z SPECIFIC FOR THIS COMBINATION OF PARAMS
        sparcityW, sparcityB, sparcityZ = 1, 1, 1
        
        # initialize W as random - to use random seed fixed for testing
        W = np.random.rand(input_dimension, projection_dimension) 
        
        # initialize protoNN
        protoNNmodel = protograph.ProtoNN(input_dimension, projection_dimension,
                                     numPrototypes, numOutputLabels,
                                     gamma=0.05, W=W)
        protoNNmodel.initializePrototypesAndGamma(train_inputs, train_targets,
                                     input_W=W, overrideGamma=True)
        protoNNtrainer = prototrainer.ProtoNNTrainer(protoNNmodel,
                                     sparcityW, sparcityB, sparcityZ,
                                     regW, regB, regZ,
                                     learningRate,
                                     lossType)
        print("Model size:", protoNNtrainer.getModelSize(), "KB")
        current_size = protoNNtrainer.getModelSize()
        result_dic = protoNNtrainer.train(
                              train_inputs.float(),
                              test_inputs.float(),
                              train_targets.float(),
                              test_targets.float(),
                              batchsize,
                              epochs,
                              printStep,
                              valStep,
                              verbose=False, history=False)
        correct = result_dic.get('correctPredictions').item()
        total  = result_dic.get('totalPredictions')
        accuracy = 100 * correct / total
        
        # Print accuracy
        print('Accuracy for fold %d with m = %f,d = %d: %d %%' % (fold, numPrototypes, projection_dimension, accuracy))
        results[fold][index_result] = accuracy
        index_result+=1
        del protoNNmodel
        del protoNNtrainer
        gc.collect()

    print('--------------------------------')

# Print fold results
print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds}')
print('--------------------------------')
results = np.sum(results, axis = 0) / k_folds
print("Average Accuracy")
for i in range (0, test_case):
    print("ProtoNN with size", size[0], "KB, prototype", structure[i][0], ", prj dim", structure[i][1],", Accuracy: \t:", results[i])

--------------------------------
FOLD 0
--------------------------------
num prototypes:	 50 
porjection dim:	 50
Model size: 63.2 KB
Accuracy for fold 0 with m = 50.000000,d = 50: 94 %
num prototypes:	 480 
porjection dim:	 15
Model size: 63.36 KB
Accuracy for fold 0 with m = 480.000000,d = 15: 88 %
num prototypes:	 240 
porjection dim:	 15
Model size: 39.36 KB
Accuracy for fold 0 with m = 240.000000,d = 15: 92 %
num prototypes:	 50 
porjection dim:	 15
Model size: 20.36 KB
Accuracy for fold 0 with m = 50.000000,d = 15: 93 %
--------------------------------
FOLD 1
--------------------------------
num prototypes:	 50 
porjection dim:	 50
Model size: 63.2 KB
Accuracy for fold 1 with m = 50.000000,d = 50: 93 %
num prototypes:	 480 
porjection dim:	 15
Model size: 63.36 KB
Accuracy for fold 1 with m = 480.000000,d = 15: 89 %
num prototypes:	 240 
porjection dim:	 15
Model size: 39.36 KB
Accuracy for fold 1 with m = 240.000000,d = 15: 93 %
num prototypes:	 50 
porjection dim:	 15
Model siz

In [10]:
# Start print
knn = KNeighborsClassifier(n_neighbors = 3)

knn_results = {}
print('--------------------------------')
# K-fold Cross Validation model evaluation
for fold, (train_ids, test_ids) in enumerate(stratifiedk_fold.split(dataset.data, Y)): # give y is just for compatibility in KFOLD

    # Print
    print(f'FOLD {fold}')
    print('--------------------------------')
    
    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
    
    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=train_ids.shape[0],
                      sampler=train_subsampler)
    testloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=test_ids.shape[0],
                      sampler=test_subsampler)

    train_inputs, train_targets = next(iter(trainloader))
    test_inputs, test_targets = next(iter(testloader))

    # K-NN
    # test KNN
    knn.fit(train_inputs, train_targets)
    knn_foldscore = knn.score(test_inputs,test_targets)
    knn_results[fold] = knn_foldscore*100
    print("KNN accuracy:",knn_foldscore)
    print('--------------------------------')

sum = 0.0
for key, value in knn_results.items():
    print(f'Fold {key}: {value} %')
    sum += value
print(f'Average KNN: {sum/len(knn_results.items())} %')

--------------------------------
FOLD 0
--------------------------------
KNN accuracy: 0.9520219328307059
--------------------------------
FOLD 1
--------------------------------
KNN accuracy: 0.9588477366255144
--------------------------------
FOLD 2
--------------------------------
KNN accuracy: 0.9602194787379973
--------------------------------
FOLD 3
--------------------------------
KNN accuracy: 0.9547325102880658
--------------------------------
FOLD 4
--------------------------------
KNN accuracy: 0.9458161865569273
--------------------------------
Fold 0: 95.20219328307059 %
Fold 1: 95.88477366255144 %
Fold 2: 96.02194787379973 %
Fold 3: 95.47325102880659 %
Fold 4: 94.58161865569274 %
Average KNN: 95.43275690078421 %


In [11]:
import winsound
# to notify that I'm done running
duration = 1000  # milliseconds
freq = 440  # Hz
winsound.Beep(freq, duration)
winsound.Beep(500, duration)